In [32]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from physoce.util import haversine
from airsea.windstress import stress
from glob import glob
import gsw
import os.path

Adapted from https://github.com/pyoceans/python-oceans/blob/master/oceans/sw_extras/sw_extras.py

Check sw_extras since latitude should be multiplied by a factor of pi/180

In [22]:
def cor_beta(lat):
    omega = 7.2921e-5
    earth_radius = 6.371e6
    return 2 * omega * np.cos(lat*np.pi/180) / earth_radius

### Load data

#### Rover

In [2]:
# Load data
rover_csv_file = 'data/Rover_II_Current_Mag_Hourly_Avg_pad_2018.csv'
df = pd.read_csv(rover_csv_file,parse_dates=[7])

# Datetime index
df = df.set_index('Date_time_R')

# Create u and v components with units m/s
df['u'] = df['Easting (cm/sec)']/100
df['v'] = df['Northing (cm/sec)']/100

#### NCEP-NARR

In [43]:
narr_dir = '/Users/tomconnolly/work/Data/NCEP-NARR/'

u_file_list = sorted(glob(os.path.join(narr_dir,'uwnd.10m.*.nc')))
v_file_list = sorted(glob(os.path.join(narr_dir,'vwnd.10m.*.nc')))

u = xr.open_mfdataset(u_file_list)
v = xr.open_mfdataset(v_file_list)

/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/tomconnolly/programs/anaconda/envs/py37/lib/python3.7/site-packages/xarray/conventions.py:362: SerializationWarning: variable 'uwnd' has multiple fill values {9.96921e+36, -9.96921e+36}, decoding all values to NaN.
  stack_char_

### Station M location

In [44]:
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

#### Find NCEP-NARR indices of nearest point to Station M

In [45]:
dist = np.array(np.sqrt(((mlat-u['lat'])*111)**2 + ((mlon-u['lon'])*np.cos(mlat*np.pi/180)*111)**2))

In [46]:
idx = np.argmin(dist,axis=None)
ii,jj = np.unravel_index(idx,np.shape(dist))

#### Compute wind stress curl

In [47]:
dx = np.array(v['x'][jj+1] - v['x'][jj-1])
dy = np.array(u['y'][ii+1] - u['y'][ii-1])

f = np.array(gsw.f(u['lat']))
rho = 1045

beta = np.array(cor_beta(u['lat'][ii,jj]))
H = 4000

Forcing term in Koblinsky et al.

In [48]:
curl_tau_F = f[ii,jj]*((1/dx)*(stress(np.squeeze(v['vwnd'][:,ii,jj+1]))/(rho*f[ii,jj+1]) - 
                           stress(np.squeeze(v['vwnd'][:,ii,jj-1]))/(rho*f[ii,jj-1])) - 
           (1/dy)*(stress(np.squeeze(u['uwnd'][:,ii+1,jj]))/(rho*f[ii+1,jj]) - 
                   stress(np.squeeze(u['uwnd'][:,ii-1,jj]))/(rho*f[ii-1,jj])))

In [49]:
u['lat'][ii,jj]

<xarray.DataArray 'lat' ()>
array(35.28079, dtype=float32)
Coordinates:
    lat      float32 35.28079
    lon      float32 -122.9061
    y        float32 3116448.0
    x        float32 4155264.0
Attributes:
    axis:                Y
    coordinate_defines:  point
    long_name:           Latitude
    standard_name:       latitude
    units:               degrees_north

In [50]:
plt.figure()
plt.plot(u['time'],np.squeeze(u['uwnd'][:,ii,jj]))
plt.plot(v['time'],np.squeeze(v['vwnd'][:,ii,jj]))

FigureCanvasNbAgg()

Wind stress curl

In [51]:
plt.figure()
plt.plot(u['time'],curl_tau_F/(beta*H))

FigureCanvasNbAgg()

In [58]:
plt.figure()
plt.plot(u['time'],curl_tau_F/(beta*H)*10)
plt.plot(df.index,df['u'])

FigureCanvasNbAgg()